In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sn

folder='project/data/'
# combining all fields' publication record data in a single file
pubrecords=pd.DataFrame()

data=pd.read_csv(folder + 'Chemistry publication record.csv',encoding="ISO-8859-1", header=0)
data['Field']='Chemistry'
pubrecords=pubrecords.append(data)

data=pd.read_csv(folder + 'Physics publication record.csv',encoding="ISO-8859-1", header=0)
data['Field']='Physics'
pubrecords=pubrecords.append(data)

data=pd.read_csv(folder + 'Medicine publication record.csv',encoding="ISO-8859-1", header=0)
data['Field']='Medicine'
pubrecords=pubrecords.append(data)


# finding the publication records before the prize announcement year
pubrecords_bfr_prz=pubrecords[pubrecords['Pub year']<pubrecords['Prize year']]

pubyear=pubrecords_bfr_prz.groupby(['Laureate ID'])['Pub year']
przyear=pubrecords_bfr_prz.groupby(['Laureate ID'])['Prize year']
pub_range=(przyear.max()-pubyear.min()+1).sort_values()


# creating distribution of time slots laureates published papers before the prize
lauraetes_distrb=[[],[]]
timeslot=5
for i in range(int(pub_range.max()/timeslot)+1):
    lauraetes_distrb[0].append(pub_range[(pub_range>(i*timeslot))&(pub_range<((i+1)*timeslot))].count())
    lauraetes_distrb[1].append(str(timeslot*i)+'-'+str(timeslot*(i+1)))
lauraetes_distrb=pd.DataFrame({'laureates_numbers':lauraetes_distrb[0], 'timeslots':lauraetes_distrb[1]})

#plotting the first figure
plt.bar(lauraetes_distrb['timeslots'],lauraetes_distrb['laureates_numbers'], align='center', alpha=1, width=0.5)
plt.tick_params(axis='x', rotation=90)
plt.xlabel('Number of Years Nobel Laureates Published Papers \n Before Getting Nobel Prize') 
plt.ylabel('Number of Nobel Laureates')
 

# for the second figure
univ_nobels_papers=pd.DataFrame()
univ_nobels_papers['University_Institute']=pubrecords['Affiliation'].value_counts().index

# find number of publications for each university/institution
univ_nobels_papers['Number_of_publications']=pubrecords['Affiliation'].value_counts().values
univ_nobels_papers=univ_nobels_papers.sort_values('University_Institute')

# find number of prizes for each university/institution
x=pubrecords.sort_values('Affiliation')
univ_nobels_papers['Number_of_Nobels']=x.groupby(['Affiliation','Prize year'])['Affiliation'].unique().value_counts().sort_index().values 
univ_nobels_papers=univ_nobels_papers.sort_values('Number_of_Nobels', ascending=False)


## plotting the second figure
p1=sn.lmplot( x="Number_of_publications", y="Number_of_Nobels", data=univ_nobels_papers, fit_reg=False, hue='University_Institute', legend=False, palette="Set1")

plt.title('Distribution of Universities by their Total Nobel Prize\n and Number of Papers Their Lauraetes Published')
# Setting x-axis label
plt.xlabel('Number of Publications')
# Setting y-axis label
plt.ylabel('Number of Nobel Prizes')
for line in range(0,10):
     plt.text(univ_nobels_papers.Number_of_publications.iloc[line]+0.2, 
     univ_nobels_papers.Number_of_Nobels.iloc[line], 
     univ_nobels_papers.University_Institute.iloc[line], horizontalalignment='left', size='medium', color='black', 
     weight='light', rotation=-11)